In [1]:
#get the probabilities

In [2]:
import pandas as pd
import os
import re
from nltk.stem.porter import *

In [3]:
data = pd.read_csv("Review_Subset.csv")

In [4]:
def getNgrams(text, n):
    if n == 1:
        return text
    
    text.append("<end>")
    ngramList = []
    for i in range(len(text) - (n-1)):
        ngram = text[i:i+n]
        ngramList.append(ngram)
    
    return ngramList

In [5]:
unigramDict = {}
bigramDict = {}
stemmer = PorterStemmer()

def getCountsinRow(row):
    text = row["text"]
    parsedText = re.findall("(\w+)", text)
    
    for index in range(len(parsedText)):
        parsedText[index] = stemmer.stem(parsedText[index])
        parsedText[index] = parsedText[index].lower()
        
    unigram = getNgrams(parsedText, 1)
    bigram = getNgrams(parsedText, 2)
    ngrams = (unigram,bigram)
    for index in range(len(ngrams)):
        for eachWord in ngrams[index]:
            
            if index == 0:
                #unigram
                
                if eachWord in unigramDict:
                    unigramDict[eachWord] +=1
                else:
                    unigramDict[eachWord] =1
                    
            if index == 1:
                #bigram
                    eachWord = tuple(eachWord)
                    if eachWord in bigramDict:
                        bigramDict[eachWord] +=1
                    else:
                        bigramDict[eachWord] =1

            

In [6]:

#get the probabilies into unigramsDict and bigramsDict
data.apply(getCountsinRow, axis = 1)

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
1970    None
1971    None
1972    None
1973    None
1974    None
1975    None
1976    None
1977    None
1978    None
1979    None
1980    None
1981    None
1982    None
1983    None
1984    None
1985    None
1986    None
1987    None
1988    None
1989    None
1990    None
1991    None
1992    None
1993    None
1994    None
1995    None
1996    None
1997    None
1998    None
1999    None
Length: 2000, dtype: object

In [7]:
print(len(unigramDict))
print(len(bigramDict))

6120
60028


In [8]:
#export unigrams
extractList = []
for eachKey in unigramDict:
    extractList.append([eachKey,unigramDict[eachKey]])

pd.DataFrame(extractList, columns = ["word", "count"]).to_csv("unigramProbabilities.csv", index = False)

In [9]:
#export bigrams
extractList = []
for eachKey in bigramDict:
    extractList.append([eachKey[0],eachKey[1],bigramDict[eachKey]])
    
pd.DataFrame(extractList, columns = ["first", "second", "count"]).to_csv("bigramProbabilities.csv", index = False)

In [10]:
print(data)

      score                                               text
0         5  I have bought several of the Vitality canned d...
1         1  Product arrived labeled as Jumbo Salted Peanut...
2         4  This is a confection that has been around a fe...
3         2  If you are looking for the secret ingredient i...
4         5  Great taffy at a great price.  There was a wid...
5         4  I got a wild hair for taffy and ordered this f...
6         5  This saltwater taffy had great flavors and was...
7         5  This taffy is so good.  It is very soft and ch...
8         5  Right now I'm mostly just sprouting this so my...
9         5  This is a very healthy dog food. Good for thei...
10        5  I don't know if it's the cactus or the tequila...
11        5  One of my boys needed to lose some weight and ...
12        1  My cats have been happily eating Felidae Plati...
13        4  good flavor! these came securely packed... the...
14        5  The Strawberry Twizzlers are my guilty ple

In [11]:
#get the vector model in data
unigramVM = []
def getUnigramReviewVectors(row):
    
    #we have data,unigrams,bigrams
    rowVector = {}
    text = row["text"]
    score = row["score"]
    parsedText = re.findall("(\w+)", text)
    for index in range(len(parsedText)):
        parsedText[index] = stemmer.stem(parsedText[index])
        parsedText[index] = parsedText[index].lower()
        score = unigramDict[parsedText[index]]
        rowVector[parsedText[index]] = score
    
    if score > 4:
        rowVector["<label>"] = "good"
    else:
        rowVector["<label>"] = "bad"
        
    unigramVM.append(rowVector)
    

data.apply(getUnigramReviewVectors, axis = 1)
    
    
    

0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
10      None
11      None
12      None
13      None
14      None
15      None
16      None
17      None
18      None
19      None
20      None
21      None
22      None
23      None
24      None
25      None
26      None
27      None
28      None
29      None
        ... 
1970    None
1971    None
1972    None
1973    None
1974    None
1975    None
1976    None
1977    None
1978    None
1979    None
1980    None
1981    None
1982    None
1983    None
1984    None
1985    None
1986    None
1987    None
1988    None
1989    None
1990    None
1991    None
1992    None
1993    None
1994    None
1995    None
1996    None
1997    None
1998    None
1999    None
Length: 2000, dtype: object

In [ ]:
pd.DataFrame(unigramVM).to_csv("UnigramVectorModel.csv", index = False)

In [ ]:
len(unigramVM)
len(unigramDict)